# Verificar el RUC de la empresa en la SUNAT usando Selenium

In [9]:
# !pip install selenium webdriver-manager

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurar opciones de Chrome
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Descomenta para modo sin interfaz gráfica
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializar el driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # Navegar a una página
    driver.get('https://e-consultaruc.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaWeb.jsp')
    
    # Esperar y encontrar el input usando su ID
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="txtRuc"]'))
    )

    # Esperar un momento para asegurarse de que la página esté lista
    time.sleep(5)

    # Escribir en el input
    input_field.send_keys("20601725551")

    # hacer click en boton de buscar
    boton_buscar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnAceptar"]'))
    )
    boton_buscar.click()
    
    # Esperar para ver los resultados
    time.sleep(5)

    # Obtener todos los h4 dentro de div.list-group
    elementos_h4 = driver.find_elements(By.CSS_SELECTOR, "div.list-group h4")
    
    # Obtener el segundo h4 (índice 1)
    if len(elementos_h4) >= 2:
        texto_elemento = elementos_h4[1]
        print("Texto del segundo h4:", texto_elemento.text)
    else:
        print("No hay suficientes elementos h4")

        
finally:
    # Cerrar el navegador
    driver.quit()

Texto del segundo h4: 20601725551 - CLINICA CERRO COLORADO S.A.C.


# Verificar el DNI de una persona en la RENIEC usando Selenium

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurar opciones de Chrome
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Descomenta para modo sin interfaz gráfica
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializar el driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # Navegar a una página
    driver.get('https://eldni.com/')
    
    # Esperar y encontrar el input del DNI
    input_dni = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="dni"]'))
    )
    
    # Esperar un momento
    time.sleep(2)
    
    # Escribir el DNI
    input_dni.send_keys("78887022")
    
    # Hacer click en el botón de buscar
    boton_buscar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-buscar-datos-por-dni"]'))
    )
    boton_buscar.click()
    
    # Esperar para ver los resultados
    time.sleep(5)
    
    # Extraer el texto del elemento samp
    texto_resultado = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="column-center"]/div[1]/div[1]/samp'))
    )
    
    print("Datos extraídos:", texto_resultado.text)
         
finally:
    # Cerrar el navegador
    driver.quit()

Datos extraídos: JORDY JOSEPH LOAYZA GIRALT


# (DESCARTADO) Verificar el NOMBRE del COMPROBANTE con un modelo de inteligencia artificial

## Probar un NOMBRE y Apellidos con NER

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from typing import List
from dataclasses import dataclass

@dataclass
class NameDetection:
    """Información de un nombre detectado"""
    text: str
    confidence: float

class NERModelConfig:
    """Configuración del modelo NER"""
    MODEL_NAME = "MMG/xlm-roberta-large-ner-spanish"
    BATCH_SIZE = 8
    CONFIDENCE_THRESHOLD = 0.0  # ✨ Cambiado a 0.0 para capturar TODO


class DeviceSelector:
    """Selecciona el mejor dispositivo disponible"""
    
    @staticmethod
    def get_best_device() -> torch.device:
        """Retorna el mejor dispositivo disponible (MPS > CUDA > CPU)"""
        if torch.backends.mps.is_available():
            return torch.device("mps")
        elif torch.cuda.is_available():
            return torch.device("cuda")
        return torch.device("cpu")


class NERModelLoader:
    """Responsable de cargar y configurar el modelo NER"""
    
    def __init__(self, model_name: str, device: torch.device):
        self.model_name = model_name
        self.device = device
        self.pipeline = self._load_model()
    
    def _load_model(self) -> pipeline:
        """Carga el modelo con optimizaciones"""
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model = AutoModelForTokenClassification.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device.type != "cpu" else torch.float32
        )
        
        model = model.to(self.device)
        model.eval()
        
        return pipeline(
            "ner",
            model=model,
            tokenizer=tokenizer,
            aggregation_strategy="simple",
            device=self.device,
            batch_size=NERModelConfig.BATCH_SIZE
        )


class TextFilter:
    """Filtra textos que obviamente no son nombres"""
    
    @staticmethod
    def is_valid_candidate(texto: str) -> bool:
        """Verifica si un texto es candidato válido para ser nombre"""
        if any(char.isdigit() for char in texto):
            return False
        
        palabras = texto.split()
        if len(palabras) < 2:
            return False
        
        if any(len(p) < 2 or len(p) > 20 for p in palabras):
            return False
        
        return True


class PersonNameClassifier:
    """Clasifica si un texto es un nombre de persona"""
    
    def __init__(self, ner_pipeline: pipeline, confidence_threshold: float = NERModelConfig.CONFIDENCE_THRESHOLD):
        self.ner_pipeline = ner_pipeline
        self.confidence_threshold = confidence_threshold
        self.text_filter = TextFilter()
    
    def classify_batch(self, textos: List[str]) -> List[NameDetection]:
        """Clasifica múltiples textos y retorna TODOS con su confianza"""
        resultados = []
        textos_validos = []
        indices_originales = []
        
        # Separar textos válidos de inválidos
        for i, texto in enumerate(textos):
            if self.text_filter.is_valid_candidate(texto):
                textos_validos.append(texto)
                indices_originales.append(i)
            else:
                # Texto inválido (tiene números, muy corto, etc.)
                resultados.append(NameDetection(
                    text=texto,
                    confidence=0.0
                ))
        
        if not textos_validos:
            return resultados
        
        # Procesar con NER los textos válidos
        with torch.no_grad():
            entidades_batch = self.ner_pipeline(textos_validos)
        
        # Crear lista temporal con resultados de textos válidos
        resultados_validos = []
        for idx, entidades in enumerate(entidades_batch):
            detection = self._get_person_entity_with_confidence(
                entidades,
                textos_validos[idx]
            )
            resultados_validos.append(detection)
        
        # Reconstruir orden original
        resultado_final = []
        idx_validos = 0
        for i, texto in enumerate(textos):
            if self.text_filter.is_valid_candidate(texto):
                resultado_final.append(resultados_validos[idx_validos])
                idx_validos += 1
            else:
                resultado_final.append(NameDetection(text=texto, confidence=0.0))
        
        return resultado_final
    
    def _get_person_entity_with_confidence(
        self, 
        entidades: List, 
        texto_original: str
    ) -> NameDetection:
        """Extrae la confianza de la entidad (siempre retorna NameDetection)"""
        max_confidence = 0.0
        
        for ent in entidades:
            if ent['entity_group'] == "PER":
                max_confidence = max(max_confidence, ent['score'])
        
        return NameDetection(
            text=texto_original,
            confidence=max_confidence
        )


class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_results(detections: List[NameDetection]):
        """Imprime TODOS los textos con su confianza"""
        for detection in detections:
            confidence_percent = detection.confidence * 100
            print(f"text: {detection.text}, confianza: {confidence_percent:.0f}%")


def main():
    # Datos de prueba
    ocr_texts = [
        "CONDORI QUISPICHITO MIGUEL ANGEL",
        "INVERSIONES RUBIN'S S.A.C",
        "41979614",
        "BOLETA DE VENTA",
        "PAXI JUCHANI FRANS EDWARD",
        "GARCIA LOPEZ JUAN CARLOS",
        "20427799973",
        "MAMANI MAMANI GIAN GROBER",
        "TOTAL A PAGAR",
    ]
    
    # Configurar dispositivo
    device = DeviceSelector.get_best_device()
    
    # Cargar modelo
    model_loader = NERModelLoader(NERModelConfig.MODEL_NAME, device)
    
    # Crear clasificador
    classifier = PersonNameClassifier(model_loader.pipeline)
    
    # Clasificar textos
    detections = classifier.classify_batch(ocr_texts)
    
    # Mostrar resultados
    ResultPrinter.print_results(detections)


if __name__ == '__main__':
    main()

Device set to use mps


text: CONDORI QUISPICHITO MIGUEL ANGEL, confianza: 85%
text: INVERSIONES RUBIN'S S.A.C, confianza: 0%
text: 41979614, confianza: 0%
text: BOLETA DE VENTA, confianza: 0%
text: PAXI JUCHANI FRANS EDWARD, confianza: 46%
text: GARCIA LOPEZ JUAN CARLOS, confianza: 83%
text: 20427799973, confianza: 0%
text: MAMANI MAMANI GIAN GROBER, confianza: 0%
text: TOTAL A PAGAR, confianza: 0%


## Probar con un COMPROBANTE con NER

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import easyocr
import time
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
from typing import List, Tuple, Optional, Union
from dataclasses import dataclass
import os

@dataclass
class NameDetection:
    """Información de un nombre detectado"""
    text: str
    confidence: float

class NERModelConfig:
    """Configuración del modelo NER"""
    MODEL_NAME = "MMG/xlm-roberta-large-ner-spanish"
    BATCH_SIZE = 8

class DeviceSelector:
    """Selecciona el mejor dispositivo disponible"""
    
    @staticmethod
    def get_best_device() -> torch.device:
        """Retorna el mejor dispositivo disponible (MPS > CUDA > CPU)"""
        if torch.backends.mps.is_available():
            return torch.device("mps")
        elif torch.cuda.is_available():
            return torch.device("cuda")
        return torch.device("cpu")

class ImageLoader:
    """Responsable de cargar imágenes desde PDF o archivos de imagen"""
    
    SUPPORTED_FORMATS = {'.pdf', '.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
    
    @classmethod
    def load_images(cls, file_path: str, dpi: int = 300) -> List:
        """Carga imágenes desde PDF o archivos de imagen"""
        file_ext = os.path.splitext(file_path)[1].lower()
        
        if file_ext not in cls.SUPPORTED_FORMATS:
            raise ValueError(f"Formato no soportado: {file_ext}. Formatos válidos: {cls.SUPPORTED_FORMATS}")
        
        if file_ext == '.pdf':
            return cls._load_from_pdf(file_path, dpi)
        else:
            return cls._load_from_image(file_path)
    
    @staticmethod
    def _load_from_pdf(pdf_path: str, dpi: int) -> List:
        """Convierte un PDF a lista de imágenes"""
        print(f"📄 Convirtiendo PDF a imágenes (DPI: {dpi})...")
        return convert_from_path(pdf_path, dpi=dpi)
    
    @staticmethod
    def _load_from_image(image_path: str) -> List:
        """Carga una imagen individual"""
        print(f"🖼️  Cargando imagen: {os.path.basename(image_path)}")
        imagen = Image.open(image_path)
        return [imagen]

class OCRProcessor:
    """Responsable de procesar OCR en imágenes"""
    
    def __init__(self, language: str = 'es'):
        """Inicializa el lector OCR"""
        print("Inicializando EasyOCR...")
        self.reader = easyocr.Reader([language])
    
    def process_images(self, imagenes: List) -> Tuple[List[str], float]:
        """Procesa todas las imágenes y retorna textos con tiempo total"""
        all_texts = []
        total_time = 0
        
        for i, imagen in enumerate(imagenes):
            if len(imagenes) > 1:
                print(f'\n--- Página {i+1}/{len(imagenes)} ---')
            else:
                print(f'\n--- Procesando imagen ---')
            
            # Convertir PIL Image a numpy array
            imagen_np = np.array(imagen)
            
            # Realizar OCR y medir tiempo
            start_time = time.time()
            resultados = self.reader.readtext(imagen_np)
            inference_time = time.time() - start_time
            total_time += inference_time
            
            # Extraer textos detectados
            page_texts = []
            for bbox, texto, confianza in resultados:
                page_texts.append(texto)
            
            print(f'Textos detectados: {len(page_texts)}')
            print(f'Tiempo de inferencia: {inference_time:.2f}s')
            all_texts.extend(page_texts)
        
        return all_texts, total_time

class NERModelLoader:
    """Responsable de cargar y configurar el modelo NER"""
    
    def __init__(self, model_name: str, device: torch.device):
        self.model_name = model_name
        self.device = device
        print(f"Cargando modelo NER en {device}...")
        self.pipeline = self._load_model()
    
    def _load_model(self) -> pipeline:
        """Carga el modelo con optimizaciones"""
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model = AutoModelForTokenClassification.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device.type != "cpu" else torch.float32
        )
        
        model = model.to(self.device)
        model.eval()
        
        return pipeline(
            "ner",
            model=model,
            tokenizer=tokenizer,
            aggregation_strategy="simple",
            device=self.device,
            batch_size=NERModelConfig.BATCH_SIZE
        )

class PersonNameClassifier:
    """Clasifica si un texto es un nombre de persona"""
    
    def __init__(self, ner_pipeline: pipeline):
        self.ner_pipeline = ner_pipeline
    
    def classify_batch(self, textos: List[str]) -> List[NameDetection]:
        """Clasifica TODOS los textos sin filtrado previo"""
        if not textos:
            return []
        
        print(f"\nClasificando {len(textos)} textos con NER...")
        
        # Procesar todos los textos directamente con NER
        with torch.no_grad():
            entidades_batch = self.ner_pipeline(textos)
        
        # Crear lista con resultados
        resultados = []
        for idx, entidades in enumerate(entidades_batch):
            detection = self._get_person_entity_with_confidence(
                entidades,
                textos[idx]
            )
            resultados.append(detection)
        
        return resultados
    
    def _get_person_entity_with_confidence(
        self, 
        entidades: List, 
        texto_original: str
    ) -> NameDetection:
        """Extrae la confianza de la entidad PER (siempre retorna NameDetection)"""
        max_confidence = 0.0
        
        for ent in entidades:
            if ent['entity_group'] == "PER":
                max_confidence = max(max_confidence, ent['score'])
        
        return NameDetection(
            text=texto_original,
            confidence=max_confidence
        )

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_ner_results(detections: List[NameDetection]):
        """Imprime resultados de clasificación NER ordenados por confianza"""
        print(f'\n{"="*50}')
        print(f'=== RESULTADOS CLASIFICACIÓN NER ===')
        print(f'=== (Ordenados de Mayor a Menor Confianza) ===')
        print(f'{"="*50}')
        for i, detection in enumerate(detections, 1):
            confidence_percent = detection.confidence * 100
            print(f"{i}. Texto: {detection.text}, Confianza: {confidence_percent:.2f}%")
        print(f'{"="*50}')

    @staticmethod
    def get_first_with_confidence(detections: List[NameDetection], min_confidence: float = 0.20) -> Optional[NameDetection]:
        """Obtiene el primer texto con confianza mayor al threshold"""
        for detection in detections:
            if detection.confidence > min_confidence:
                return detection
        return None

class ComprobanteNameProcessor:
    """Orquestador principal del procesamiento de nombres en comprobantes"""
    
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.image_loader = ImageLoader()
        self.ocr_processor = OCRProcessor()
        self.result_printer = ResultPrinter()
        
        # Configurar dispositivo y cargar modelo NER
        device = DeviceSelector.get_best_device()
        model_loader = NERModelLoader(NERModelConfig.MODEL_NAME, device)
        self.classifier = PersonNameClassifier(model_loader.pipeline)
    
    def process(self):
        """Procesa el archivo completo: Carga + OCR + Clasificación NER"""
        # 1. Cargar imágenes (desde PDF o archivo de imagen)
        imagenes = self.image_loader.load_images(self.file_path)
        
        # 2. Procesar OCR
        all_texts, ocr_time = self.ocr_processor.process_images(imagenes)
        
        # 3. Clasificar nombres con NER
        detections = self.classifier.classify_batch(all_texts)
        
        # 4. Imprimir resultados NER
        self.result_printer.print_ner_results(detections)
        
        # 5. Imprimir el primer nombre con confianza > 20%
        primer_nombre = self.result_printer.get_first_with_confidence(detections)
        
        if primer_nombre:
            confidence_percent = primer_nombre.confidence * 100
            print(f'\n🎯 PRIMER NOMBRE CON CONFIANZA > 20%:')
            print(f'   Texto: {primer_nombre.text}')
            print(f'   Confianza: {confidence_percent:.2f}%')
        else:
            print('\n⚠️  No se encontró ningún texto con confianza mayor a 20%')
        
        print(f'\n⏱️  Tiempo total OCR: {ocr_time:.2f}s')
        
        return detections

def main():
    file_path = 'comprobante/WhatsApp Image 2025-10-16 at 21.24.34.jpeg'
    
    processor = ComprobanteNameProcessor(file_path)
    detections = processor.process()

if __name__ == '__main__':
    main()

Inicializando EasyOCR...
Cargando modelo NER en mps...


Device set to use mps


🖼️  Cargando imagen: WhatsApp Image 2025-10-16 at 21.24.34.jpeg

--- Procesando imagen ---


/opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Textos detectados: 91
Tiempo de inferencia: 1.90s

Clasificando 91 textos con NER...

=== RESULTADOS CLASIFICACIÓN NER ===
=== (Ordenados de Mayor a Menor Confianza) ===
1. Texto: Mifarnag8", Confianza: 0.00%
2. Texto: NiFARhA S,A €, Confianza: 0.00%
3. Texto: RUC: 20512002090, Confianza: 0.00%
4. Texto: CENTRAL:, Confianza: 0.00%
5. Texto: Cal ., Confianza: 0.00%
6. Texto: Victor, Confianza: 0.00%
7. Texto: Alzamor a, Confianza: 68.87%
8. Texto: Tro, Confianza: 0.00%
9. Texto: T77, Confianza: 0.00%
10. Texto: Urb, Confianza: 0.00%
11. Texto: Santa, Confianza: 0.00%
12. Texto: Catalina, Confianza: 90.41%
13. Texto: La Victoria, Confianza: 0.00%
14. Texto: Lima TELF, Confianza: 0.00%
15. Texto: 2130760, Confianza: 0.00%
16. Texto: Tienoa €55, Confianza: 0.00%
17. Texto: JULIAcA SaN RoHAN 3, Confianza: 0.00%
18. Texto: 1052, Confianza: 0.00%
19. Texto: Jr, Confianza: 0.00%
20. Texto: San Roman Nro, 521 Puno, Confianza: 0.00%
21. Texto: San Ronan, Confianza: 0.00%
22. Texto: Juhiaca, Conf